In [1]:
## Memorializing this as the easy way to lookup accounts


In [1]:
"""
Main Entry Point
"""
from pathlib import Path
from typing import List
import argparse
from datetime import datetime

import pandas as pd # type: ignore

import src.fetch_data # type: ignore
import src.core_transform # type: ignore
import cdutils.pkey_sqlite # type: ignore
import cdutils.hhnbr # type: ignore


data = src.fetch_data.fetch_data()

# # # Core transformation pipeline
raw_data = src.core_transform.main_pipeline(data)

# Raw data with pkey appended
raw_data = cdutils.pkey_sqlite.add_pkey(raw_data)
raw_data = cdutils.pkey_sqlite.add_ownership_key(raw_data)
raw_data = cdutils.pkey_sqlite.add_address_key(raw_data)

In [2]:
househldacct = data['househldacct'].copy()
raw_data = cdutils.hhnbr.add_hh_nbr(raw_data, househldacct)

In [3]:
raw_data

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint
0,2025-06-27,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,1.593482e+09,0.0,0.0,1.593482e+09,0.0,41512,43261.0,44260.0,187340.0,2020-02-26 23:28:03
1,2025-06-27,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.065722,...,2.900000e+07,0.0,0.0,2.900000e+07,65500000.0,57110,59661.0,61147.0,193906.0,2023-09-22 08:01:38
2,2025-06-27,27071460,BRISTOL COUNTY SAVINGS BANK,Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,2.698885e+07,0.0,0.0,2.698885e+07,0.0,1221,1352.0,41896.0,255924.0,2022-09-27 22:46:27
3,2025-06-27,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,2.600000e+07,0.0,0.0,2.600000e+07,26000000.0,79218,83546.0,85431.0,288455.0,2024-06-11 12:02:04
4,2025-06-27,150988429,CITY OF ATTLEBORO,ICS Shadow - Muni - Demand,0.0,None,CK,CK36,ACT,0.000000,...,2.558304e+07,0.0,0.0,2.558304e+07,0.0,5105,35118.0,35620.0,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89811,2025-06-27,27047636,BRISTOL COUNTY SAVINGS BANK,Payoff Overages,0.0,None,BKCK,BTC2,ACT,0.000000,...,-2.098590e+03,0.0,0.0,-2.098590e+03,0.0,1221,1352.0,41645.0,255924.0,2022-09-27 22:46:27
89812,2025-06-27,151009810,PROVIDENCE HAUS MANAGEMENT LTD,Simple Business Checking,5000.0,None,CK,CK25,ACT,0.000000,...,-4.222290e+03,0.0,0.0,-4.222290e+03,5000.0,69709,81879.0,74852.0,NaN,NaT
89813,2025-06-27,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,-7.325560e+04,0.0,0.0,-7.325560e+04,0.0,1221,1352.0,41577.0,255924.0,2022-09-27 22:46:27
89814,2025-06-27,27020568,BRISTOL COUNTY SAVINGS BANK,PDO Checks - OLD,0.0,None,BKCK,BTC1,ACT,0.000000,...,-2.041281e+05,0.0,0.0,-2.041281e+05,0.0,1221,1352.0,41437.0,255924.0,2022-09-27 22:46:27


In [4]:
import cdutils.loans.calculations

In [5]:
loan_category_df = cdutils.loans.calculations.categorize_loans(raw_data)

In [6]:
loan_category_df = loan_category_df[['acctnbr','Category']].copy()

In [14]:
df = pd.merge(raw_data, loan_category_df, on='acctnbr', how='left')

In [15]:
df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category
0,2025-06-27,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,0.0,0.0,1.593482e+09,0.0,41512,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN
1,2025-06-27,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.065722,...,0.0,0.0,2.900000e+07,65500000.0,57110,59661.0,61147.0,193906.0,2023-09-22 08:01:38,CRE
2,2025-06-27,27071460,BRISTOL COUNTY SAVINGS BANK,Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,0.0,0.0,2.698885e+07,0.0,1221,1352.0,41896.0,255924.0,2022-09-27 22:46:27,NaN
3,2025-06-27,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,0.0,0.0,2.600000e+07,26000000.0,79218,83546.0,85431.0,288455.0,2024-06-11 12:02:04,CRE
4,2025-06-27,150988429,CITY OF ATTLEBORO,ICS Shadow - Muni - Demand,0.0,None,CK,CK36,ACT,0.000000,...,0.0,0.0,2.558304e+07,0.0,5105,35118.0,35620.0,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89811,2025-06-27,27047636,BRISTOL COUNTY SAVINGS BANK,Payoff Overages,0.0,None,BKCK,BTC2,ACT,0.000000,...,0.0,0.0,-2.098590e+03,0.0,1221,1352.0,41645.0,255924.0,2022-09-27 22:46:27,NaN
89812,2025-06-27,151009810,PROVIDENCE HAUS MANAGEMENT LTD,Simple Business Checking,5000.0,None,CK,CK25,ACT,0.000000,...,0.0,0.0,-4.222290e+03,5000.0,69709,81879.0,74852.0,NaN,NaT,NaN
89813,2025-06-27,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,0.0,0.0,-7.325560e+04,0.0,1221,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN
89814,2025-06-27,27020568,BRISTOL COUNTY SAVINGS BANK,PDO Checks - OLD,0.0,None,BKCK,BTC1,ACT,0.000000,...,0.0,0.0,-2.041281e+05,0.0,1221,1352.0,41437.0,255924.0,2022-09-27 22:46:27,NaN


In [9]:
import cdutils.inactive_date

df = cdutils.inactive_date.append_inactive_date(df)

In [10]:
df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category,inactivedate
0,2025-06-27,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,0.0,1.593482e+09,0.0,41512,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN,NaT
1,2025-06-27,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.065722,...,0.0,2.900000e+07,65500000.0,57110,59661.0,61147.0,193906.0,2023-09-22 08:01:38,CRE,NaT
2,2025-06-27,27071460,BRISTOL COUNTY SAVINGS BANK,Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,0.0,2.698885e+07,0.0,1221,1352.0,41896.0,255924.0,2022-09-27 22:46:27,NaN,NaT
3,2025-06-27,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,0.0,2.600000e+07,26000000.0,79218,83546.0,85431.0,288455.0,2024-06-11 12:02:04,CRE,NaT
4,2025-06-27,150988429,CITY OF ATTLEBORO,ICS Shadow - Muni - Demand,0.0,None,CK,CK36,ACT,0.000000,...,0.0,2.558304e+07,0.0,5105,35118.0,35620.0,NaN,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89811,2025-06-27,27047636,BRISTOL COUNTY SAVINGS BANK,Payoff Overages,0.0,None,BKCK,BTC2,ACT,0.000000,...,0.0,-2.098590e+03,0.0,1221,1352.0,41645.0,255924.0,2022-09-27 22:46:27,NaN,NaT
89812,2025-06-27,151009810,PROVIDENCE HAUS MANAGEMENT LTD,Simple Business Checking,5000.0,None,CK,CK25,ACT,0.000000,...,0.0,-4.222290e+03,5000.0,69709,81879.0,74852.0,NaN,NaT,NaN,NaT
89813,2025-06-27,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,0.0,-7.325560e+04,0.0,1221,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN,NaT
89814,2025-06-27,27020568,BRISTOL COUNTY SAVINGS BANK,PDO Checks - OLD,0.0,None,BKCK,BTC1,ACT,0.000000,...,0.0,-2.041281e+05,0.0,1221,1352.0,41437.0,255924.0,2022-09-27 22:46:27,NaN,NaT


In [41]:
result = df[(df['Category'].isin(['Residential']) | (df['currmiaccttypcd'].isin([
    'CM07',
    'CM08',
    'CM81',
    'CM40',
    'CM53',
    'CM54',
    'CM78'
])))].copy()

In [42]:
result['Category'].unique()

array(['CRE', 'Residential', 'C&I'], dtype=object)

In [43]:
result['curracctstatcd'].unique()

array(['ACT', 'NPFM'], dtype=object)

In [33]:
result['product'].unique()

array(['Commercial Mortgages', 'CML Fixed Construction',
       'CML ARM Construction', 'ARC ARM Loan', 'Construction to Perm',
       '25 Year Fixed Rate', '7/6 30 DAY AVERAGE SOFR', '5/1 Year ARM',
       '30 Year Fixed Rate', '20 Year Fixed Rate', 'Interest Only HELOC',
       '5/6 30 Day Average SOFR', '5/6 Construction',
       'Fixed Rate Construction', '10/6 30 Day Average SOFR',
       '15 Year Fixed Rate', 'Fixed Rate FTHB', 'Special HELOC',
       '7/1 Year ARM', '5/5 ARM FTHB', '3/1 Year ARM',
       '3/6 30 DAY AVERAGE SOFR', 'SBA Commercial Mortgages',
       '10/6 FNMA/FHLMC 30 DAY AVGSOFR', '10 Year Fixed Rate',
       'Equity Line of Credit', 'Premium Home Equity Loan',
       'FNB - Home Equity LOC', 'Equity Loans Second Mortgages',
       '3/6 LAND 30 DAY AVERAGE SOFR', 'Land Loans', 'Equity Builder AHP',
       'SBA Commercial Second Mortgage', 'FNB - Adjustable Rate Loan',
       'Serviced by Others', 'BCSB Down Payment Assistance ',
       '1 Year ARM', 'Equity LOC

In [34]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5260 entries, 1 to 89434
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   effdate                 5260 non-null   datetime64[ns]
 1   acctnbr                 5260 non-null   object        
 2   ownersortname           5260 non-null   object        
 3   product                 5260 non-null   object        
 4   noteopenamt             5260 non-null   float64       
 5   ratetypcd               5260 non-null   object        
 6   mjaccttypcd             5260 non-null   object        
 7   currmiaccttypcd         5260 non-null   object        
 8   curracctstatcd          5260 non-null   object        
 9   noteintrate             5260 non-null   float64       
 10  bookbalance             5260 non-null   float64       
 11  notebal                 5260 non-null   float64       
 12  contractdate            5260 non-null   datetime64[n

In [35]:
result = result[[
    'acctnbr',
    'ownersortname',
    'mjaccttypcd',
    'currmiaccttypcd',
    'product',
    'notebal',
    'creditlimitamt'
]].copy()

In [36]:
result

,acctnbr,ownersortname,mjaccttypcd,currmiaccttypcd,product,notebal,creditlimitamt
1,150936915,REDBROOK APARTMENTS LLC,CML,CM40,Commercial Mortgages,65500000.00,0.0
3,151058057,NBPIV SARATOGA LLC,CML,CM40,Commercial Mortgages,26000000.00,0.0
9,150862011,"R3 PROJECT COMPANY, LLC",CML,CM07,CML Fixed Construction,0.00,20000000.0
10,151038843,"POWER 250, LLC",CML,CM08,CML ARM Construction,3180000.00,27500000.0
12,150809211,29 CENTER STREET LLC,CML,CM40,Commercial Mortgages,17400593.32,0.0
...,...,...,...,...,...,...,...
89430,970141021,"ARNOLD, NORMAN S.",MTG,MG60,Equity Line of Credit,0.00,0.0
89431,970149281,"HUDSON, VICTORIA A.",MTG,MG60,Equity Line of Credit,0.00,0.0
89432,970158171,"NOISEUX, KENNETH E.",MTG,MG52,Special HELOC,0.00,0.0
89433,970186001,"SHURTLEFF, GIDEON N.",MTG,MG52,Special HELOC,0.00,0.0


In [23]:
import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore

In [24]:
def get_state_data():
    """
    Main data query
    """
    wh_acctcommon = text(f"""
    SELECT
        a.ACCTNBR,
        a.PRIMARYOWNERSTATE
    FROM
        OSIBANK.WH_ACCTCOMMON a
    """)

    queries = [
        {'key':'wh_acctcommon', 'sql':wh_acctcommon, 'engine':1},
    ]


    data = cdutils.database.connect.retrieve_data(queries)
    return data
    

In [25]:
state_data = get_state_data()

In [26]:
state_data = state_data['wh_acctcommon'].copy()

In [27]:
import cdutils.input_cleansing # type: ignore

In [28]:
state_schema = {
    'acctnbr':'str'
}

state_data = cdutils.input_cleansing.enforce_schema(state_data, state_schema)

In [37]:
result_with_state = pd.merge(result, state_data, on='acctnbr', how='left')

In [38]:
result_with_state

,acctnbr,ownersortname,mjaccttypcd,currmiaccttypcd,product,notebal,creditlimitamt,primaryownerstate
0,150936915,REDBROOK APARTMENTS LLC,CML,CM40,Commercial Mortgages,65500000.00,0.0,MA
1,151058057,NBPIV SARATOGA LLC,CML,CM40,Commercial Mortgages,26000000.00,0.0,MA
2,150862011,"R3 PROJECT COMPANY, LLC",CML,CM07,CML Fixed Construction,0.00,20000000.0,MA
3,151038843,"POWER 250, LLC",CML,CM08,CML ARM Construction,3180000.00,27500000.0,MA
4,150809211,29 CENTER STREET LLC,CML,CM40,Commercial Mortgages,17400593.32,0.0,MA
...,...,...,...,...,...,...,...,...
5255,970141021,"ARNOLD, NORMAN S.",MTG,MG60,Equity Line of Credit,0.00,0.0,MA
5256,970149281,"HUDSON, VICTORIA A.",MTG,MG60,Equity Line of Credit,0.00,0.0,MA
5257,970158171,"NOISEUX, KENNETH E.",MTG,MG52,Special HELOC,0.00,0.0,RI
5258,970186001,"SHURTLEFF, GIDEON N.",MTG,MG52,Special HELOC,0.00,0.0,MA


In [ ]:
result

In [ ]:
deposits = df[~df['acctofficer'].isnull()]


In [15]:
loans = df[~df['Category'].isnull()]

In [17]:
loans['Total Exposure'].sum()

np.float64(3058547743.898)

In [14]:
deposits['Net Balance'].sum()

np.float64(2547823557.2200003)

In [ ]:
deposits

In [12]:
df['taxrptfororgnbr'].nunique()

6274

In [13]:
wh_pers = data['wh_pers'].copy()

In [15]:
wh_pers

,persnbr,persname,perssortname,taxid,adddate,datebirth,datedeath,age,employeeyn,privacyyn,...,sicsubdesc,creditscore,spousepersnbr,spousepersname,spouseperssortname,datelastmaint,rundate,allowpromoyn,homeemail,busemail
0,1001691,LORRAINE BANDONI,"BANDONI, LORRAINE",97D759C3EC194BB39C35008F0E10A562,2013-04-01,1959-05-15,NaT,66.0,N,N,...,None,None,NaN,None,None,2025-06-11 21:12:35,2025-06-11,N,imbandoni@aol.com,None
1,1001692,RICHARD A. DAMON,"DAMON, RICHARD A.",F6F454ED79EA550FCE80E7EE3C133C08,2013-04-01,1957-03-21,NaT,68.0,N,N,...,None,None,NaN,None,None,2025-06-11 21:12:35,2025-06-11,N,richdamon@cox.net,None
2,1001693,LILIANA LOPES,"LOPES, LILIANA",51D62AE58A623E0C0C4725E4DB97F049,2013-04-01,1972-01-27,NaT,53.0,N,N,...,None,None,NaN,None,None,2025-06-11 21:12:35,2025-06-11,N,None,None
3,1001694,PAULA E. ISENBERG,"ISENBERG, PAULA E.",5C6540C2CBE783E893108BF88F926A78,2013-04-01,1953-10-10,NaT,71.0,N,N,...,None,None,NaN,None,None,2025-06-11 21:12:35,2025-06-11,N,None,None
4,1001695,ROBERT H. KENNEDY,"KENNEDY, ROBERT H.",9A30D8B855055AE7AB80D96470757426,2013-04-02,1970-08-16,NaT,54.0,N,N,...,None,None,NaN,None,None,2025-06-11 21:12:35,2025-06-11,N,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166541,1173270,STEPHANIE A. CUTTER,"CUTTER, STEPHANIE A.",None,2025-04-30,1968-10-22,NaT,56.0,N,N,...,None,None,NaN,None,None,2025-06-11 21:12:50,2025-06-11,N,None,None
166542,1173291,DEBORAH B. HALL,"HALL, DEBORAH B.",71355A024C032D378FDE2D33BADC6249,2025-05-01,1958-04-30,NaT,67.0,N,N,...,None,None,NaN,None,None,2025-06-11 21:12:50,2025-06-11,N,debbhall139@yahoo.com,None
166543,1173300,JAMES C. CHAMBERS,"CHAMBERS, JAMES C.",3F2CFC90401A57EEBA182B44BDFF36F1,2025-05-02,1983-03-02,NaT,42.0,N,N,...,None,789,NaN,None,None,2025-06-11 21:12:50,2025-06-11,N,JIMI_CHAMBERS@HOTMAIL.COM,None
166544,1173394,JANINE D. HART,"HART, JANINE D.",7655FB9FCACCC53BAEA36F887662EFC5,2025-05-06,1987-09-27,NaT,37.0,N,N,...,None,None,NaN,None,None,2025-06-11 21:12:50,2025-06-11,N,J9HART87@GMAIL.COM,None
